In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
!pip install transformers

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
skincare = pd.read_csv(f"{DATA_PATH}아모레크롤링_스킨케어_완료.csv")
skincare

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,0103*******,5,2022.09.27,40대,여성,지성,모공,남편이 극지성이에요. 원래 수퍼화산송이 미셀라만 꾸준히 사용해 오다가 지성 전용이라...
1,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,lsm2***,5,2022.08.15,40대,여성,건성,건조함,여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 ...
2,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,gpdl***,5,2022.12.24,20대,여성,복합성,트러블,이니스프리 폼클렌징은 피부에 자극적이지 않아 좋아요 선크림만 사용해도 꼭 폼까지 써...
3,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,zoll***,5,2022.09.30,30대,여성,건성,건조함,저희 엄마가 이 제품이 클렌징이 제일 잘된다고 좋아하서 1+1 이벤트하길래 4개 쟁...
4,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,snow****,5,2022.09.30,50대 이상,여성,지성,모공,여름이라서 모공이 넓어지는것 같아서 구매해서 사용해보니 촉촉하고 피부땡기지 않아서 ...
...,...,...,...,...,...,...,...,...,...,...,...
112742,스킨케어,프리메라,모이스처 클렌징 티슈 300g,drea****,5,2020.01.11,40대,여성,복합성,모공,자극적이지 않고 순하게 화장을 지워주어 좋아요.
112743,스킨케어,프리메라,모이스처 클렌징 티슈 300g,kim4***,5,2022.05.11,50대 이상,여성,복합성,칙칙함,순하고 화장도 잘지워지고 기름지지않아서 좋아요 ~~^^
112744,스킨케어,프리메라,모이스처 클렌징 티슈 300g,hyej*****,4,2022.04.12,40대,여성,수분부족지성,탄력없음,클렌징티슈는 이것만 사용해요 자극 덜하고 향도 좋아요~
112745,스킨케어,프리메라,모이스처 클렌징 티슈 300g,sooh*****,4,2019.08.31,40대,여성,수분부족지성,모공,제 인생템 중 하나입니다 없으면 불안해져 쟁여놓고 쓴답니다~


In [6]:
makeup = pd.read_csv(f"{DATA_PATH}아모레크롤링_메이크업_완료.csv")
makeup

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,메이크업,헤라,블랙 쿠션 SPF34/PA++,clas********,5,2022.05.15,30대,여성,지성,칙칙함,신랑의 인생템이예요. 전에 체험단 이벤트로 27n1컬러를 처음 만났는데 21호 쓰는...
1,메이크업,헤라,블랙 쿠션 SPF34/PA++,drea*****,5,2022.02.06,50대 이상,여성,지성,트러블,명불허전 헤라 블랙쿠션!! 왜 입소문이 자자한지 써보면 압니다. 이번에 리뉴얼 되었...
2,메이크업,헤라,블랙 쿠션 SPF34/PA++,tlsd********,5,2022.10.10,30대,여성,복합성,민감성,민감여드름 복합성피부 30대여자입니다. 블랙쿠션은 이름만 많이들었지 구매한건 이번이...
3,메이크업,헤라,블랙 쿠션 SPF34/PA++,dbsd********,5,2021.05.11,20대,여성,복합성,민감성,사실 처음엔 가격때문에 살까말까 고민이 많았던 제품이었어요. 다들 좋다고는 하는데 ...
4,메이크업,헤라,블랙 쿠션 SPF34/PA++,mksa******,5,2022.01.15,40대,여성,건성,칙칙함,피부는 밝은편인데 악건성에 칙칙 노란끼 홍조 있어요 나이가 들어서 무조건 커버력이라...
...,...,...,...,...,...,...,...,...,...,...,...
37260,메이크업,세잔느,내추럴치크N 3.5g,ghkd*****,5,2019.05.07,30대,여성,수분부족지성,주름,사랑스런 라벤더핑크 우체국택배로 작은 상자로 왔어요. 상자를 뜯어보니 비닐에 싸여 ...
37261,메이크업,세잔느,내추럴치크N 3.5g,true*****,4,2019.05.06,40대,여성,수분부족지성,모공,여리여리한 발색 제 품의 설명이 한국어로 된건 비닐 포장 밖에 적혀있어요. 그러다 ...
37262,메이크업,세잔느,내추럴치크N 3.5g,kaec****,5,2019.05.02,30대,여성,수분부족지성,칙칙함,색상이 많이 진할까 걱정했는데 조심히 바르면 아주 이뻐요!!!
37263,메이크업,세잔느,내추럴치크N 3.5g,wjdg*******,4,2019.04.27,20대,여성,복합성,건조함,발색짱짱 뚜둥!ㅎㅎ 드디어 왔습니다 ㅎㅎ 두근두근 하늘아래 같은색은 없죠? 색이 영...


In [7]:
food = pd.read_csv(f"{DATA_PATH}뷰티푸드 별점.csv")
food

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,뷰티푸드,바이탈뷰티,메타그린 부스터샷 7일,mimi**,5,2023.08.24,30대,여성,건성,건조함,여름휴가철을 앞두고 몸매관리에 필요성을 느껴서 어디껄 살까 고민하다가 마침 전에 증...
1,뷰티푸드,바이탈뷰티,메타그린 부스터샷 7일,yooy****,5,2023.08.22,40대,여성,복합성,모공,아직 복용전 입니다. 메타그린 먹고 있어서 함께 사용하려고 구입 했습니다.
2,뷰티푸드,바이탈뷰티,메타그린 부스터샷 7일,n199**,5,2023.08.21,30대,여성,복합성,트러블,아무래도 가격이 있는 편이라 좀 부담스럽긴 하지만 원래 제품을 꾸준히 먹고 도움 받...
3,뷰티푸드,바이탈뷰티,메타그린 부스터샷 7일,n199**,4,2023.08.21,30대,여성,복합성,트러블,저는 운동이랑 식단 병행하면서 도움 받았어요. 유지어터라 그럴 수도 있지만 살을 빼...
4,뷰티푸드,바이탈뷰티,메타그린 부스터샷 7일,dlfw*****,4,2023.08.19,30대,여성,건성,건조함,먹은지 며칠 안돼서 효과는 잘 모르겠으나 먹기 간편해서 좋고 맛있네요ㅎㅎ
...,...,...,...,...,...,...,...,...,...,...,...
33057,뷰티푸드,바이탈뷰티,프로바이오틱스 골드,iamj***,5,2022.03.14,40대,남성,중성,건조함,[바이탈뷰티] [프리미엄 라인] 프로바이오 골드(30일) 프로바이오 골드 30일
33058,뷰티푸드,바이탈뷰티,프로바이오틱스 골드,daki**,5,2020.05.18,40대,여성,NaN,NaN,유산균 찾아보다가 알게된상품 인데 좋은듯 합니다.
33059,뷰티푸드,바이탈뷰티,프로바이오틱스 골드,grea*******,5,2019.09.06,30대,남성,NaN,NaN,항상 즐거운 마음으로 먹고 있습니다. 인생템 짱입니다!!
33060,뷰티푸드,바이탈뷰티,프로바이오틱스 골드,ylov****,4,2019.09.06,40대,여성,NaN,NaN,제품 바뀌고나서는 증정품먹어보고 구매합니다.먹고 호과좋왔음 좋겠네요.


In [8]:
perfume = pd.read_csv(f"{DATA_PATH}향수 별점.csv")
perfume

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,향수,에스쁘아,케익 프레그런스 2.5g,dmswls89,5,2022.10.01,30대,여성,NaN,NaN,구매시점이었던 여름부터 아주 잘 사용하고 있습니다! 향은 에스쁘아 특유의 꽃향기 같...
1,향수,에스쁘아,케익 프레그런스 2.5g,dmswls89,5,2022.12.23,30대,여성,NaN,NaN,외출할 때마다 항상 사용하는 고체향수입니다! 한 달 전 재구매한 것은 아직 안뜯었고...
2,향수,에스쁘아,케익 프레그런스 2.5g,dmswls89,5,2022.08.25,30대,여성,NaN,NaN,"예전에, 정말 예~전에 에스쁘아 고체 향수 써보고 오랜만에 구입해봤습니다. 에스쁘아..."
3,향수,에스쁘아,케익 프레그런스 2.5g,Good******,5,2022.06.15,50대 이상,여성,NaN,NaN,제품을받고 제일먼저 매우 좋다는 탄성이 나오게 됩니다.조금도 아쉬움이 없는 예쁘고 ...
4,향수,에스쁘아,케익 프레그런스 2.5g,shin******,5,2022.11.05,50대 이상,여성,NaN,NaN,저렇게 이렇게 노력하고 또 노력해서 모으고 모은 뷰티포인트와 기프트카드로 수많은 제...
...,...,...,...,...,...,...,...,...,...,...,...
2860,향수,구딸,스 수와 우 자메 EDP,ymon*,5,2021.06.30,30대,여성,NaN,NaN,전형적인 장미향이에요. 요즘 유행하는 풀냄새까지 섞인 장미향도 아니고 로스폼퐁처럼 ...
2861,향수,구딸,스 수와 우 자메 EDP,empt*****,5,2022.12.14,30대,여성,NaN,NaN,"장미향은 처음인데 매우 마음에 듭니다 잎, 줄기의 풀 냄새가 섞이지 않은 장미꽃잎만..."
2862,향수,구딸,스 수와 우 자메 EDP,나나낭니,5,2022.09.14,30대,여성,NaN,NaN,장미향은 많은데 달달하지 않은 향 찾고 있었거든요. 디퓨저로 써도 좋을듯한 향이...
2863,향수,구딸,스 수와 우 자메 EDP,saga****,5,2022.06.27,30대,여성,NaN,NaN,"머스키한 센슈얼함이 있어, 선선한 계절에 잘 어울려요. 따끈한 날씨에 뿌리면 술에 ..."


In [9]:
thing = pd.read_csv(f"{DATA_PATH}소품도구 별점.csv")
thing

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,소품도구,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,ogy4***,2,2021-10-29,20대,여성,NaN,NaN,-디자인 *5점 만점 기준* -디자인 : 퍼프 위의 가죽 느낌으로 고급스러움과 ...
1,소품도구,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,bjhd**,5,2022-02-17,30대,남성,NaN,NaN,좋아요! 최고에요!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...
2,소품도구,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,kitt*****,5,2022-07-19,20대,여성,NaN,NaN,퍼프는 빨아서 사용하는 것도 찝찝하고 블랙쿠션에는 블랙쿠션 퍼프가 딱이라고 생각해서...
3,소품도구,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,sjji****,5,2021-08-14,50대 이상,여성,NaN,NaN,퍼프가 발림성이 정말로 쫀쫀해요 쫀쫀 ! 두 개의 퍼프가 비닐과 pvc백에 이중 포...
4,소품도구,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,wlsg********,5,2022-07-20,30대,여성,NaN,NaN,"밀착력에 짱 ! 뭉침없이 발라지는 퍼프, 저는 위생상 2주일에 한번씩 직접 퍼프를 ..."
...,...,...,...,...,...,...,...,...,...,...,...
17837,소품도구,피카소,103A 파우더,suy*,5,2019-08-07,20대,여성,NaN,NaN,명불허전이에요~ 파우더나 블러셔 용도로 매우 잘쓰고 있어요!
17838,소품도구,피카소,103A 파우더,yyou****,5,2019-06-25,20대,여성,NaN,NaN,부드럽고 파우더 바르기 좋아요! 다른 브러쉬로 바르면 파우더가 조금 떠서 피카소 브...
17839,소품도구,피카소,103A 파우더,amil****,5,2019-05-18,40대,여성,NaN,NaN,피카소브러쉬 너무좋아요 계속사게되요 모질도 너무 부드럽고 ~ 이번에 행사해서 할인...
17840,소품도구,피카소,103A 파우더,vbgu*****,4,2019-04-30,30대,여성,NaN,NaN,가격은 좀 비싸지만 생각보다 괜찮은것 같아요 배송은 하루만에 받았구요 보호케이스가 ...


In [10]:
frames = [skincare, makeup, food, perfume, thing]
df = pd.concat(frames, ignore_index=True)
df

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,0103*******,5,2022.09.27,40대,여성,지성,모공,남편이 극지성이에요. 원래 수퍼화산송이 미셀라만 꾸준히 사용해 오다가 지성 전용이라...
1,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,lsm2***,5,2022.08.15,40대,여성,건성,건조함,여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 ...
2,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,gpdl***,5,2022.12.24,20대,여성,복합성,트러블,이니스프리 폼클렌징은 피부에 자극적이지 않아 좋아요 선크림만 사용해도 꼭 폼까지 써...
3,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,zoll***,5,2022.09.30,30대,여성,건성,건조함,저희 엄마가 이 제품이 클렌징이 제일 잘된다고 좋아하서 1+1 이벤트하길래 4개 쟁...
4,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,snow****,5,2022.09.30,50대 이상,여성,지성,모공,여름이라서 모공이 넓어지는것 같아서 구매해서 사용해보니 촉촉하고 피부땡기지 않아서 ...
...,...,...,...,...,...,...,...,...,...,...,...
203776,소품도구,피카소,103A 파우더,suy*,5,2019-08-07,20대,여성,NaN,NaN,명불허전이에요~ 파우더나 블러셔 용도로 매우 잘쓰고 있어요!
203777,소품도구,피카소,103A 파우더,yyou****,5,2019-06-25,20대,여성,NaN,NaN,부드럽고 파우더 바르기 좋아요! 다른 브러쉬로 바르면 파우더가 조금 떠서 피카소 브...
203778,소품도구,피카소,103A 파우더,amil****,5,2019-05-18,40대,여성,NaN,NaN,피카소브러쉬 너무좋아요 계속사게되요 모질도 너무 부드럽고 ~ 이번에 행사해서 할인...
203779,소품도구,피카소,103A 파우더,vbgu*****,4,2019-04-30,30대,여성,NaN,NaN,가격은 좀 비싸지만 생각보다 괜찮은것 같아요 배송은 하루만에 받았구요 보호케이스가 ...


In [11]:
def chatbot(df):
    model_name = "noahkim/KoT5_news_summarization"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        # 여러 키워드에 대한 처리
        keywords = title.split()
        condition = lambda x: all(keyword.lower() in x.lower() for keyword in keywords)
        # 중복 제거를 위해 unique() 사용
        matching_titles = [t for t in df['상품명'].unique().tolist() if condition(t)]
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")
        # 사용자가 상품을 선택하도록 함
        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
        selected_title = matching_titles[selected_idx - 1]
        # 선택한 상품의 리뷰 가져오기
        reviews = df[df['상품명'] == selected_title]['리뷰'].to_list()
        reviews_text = ' '.join(reviews)
        # 리뷰 요약
        inputs = tokenizer(reviews_text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=40, max_length=200, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print(f"{selected_title}의 전체 리뷰 요약: {summary}")

In [12]:
chatbot(df)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 구딸
해당 상품이 없습니다.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 스 수와
일치하는 상품명 목록:
1. 스 수와 우 자메 EDP
상품을 선택하세요 (번호 입력): 1
스 수와 우 자메 EDP의 전체 리뷰 요약: 스수와우자메는 구딸 최고의 역작으로 꼽힌다는데, 누구나 좋아할 장미향은 아닌 거 같지만 스수와우자메의 매력에 빠지면 대체불가지로 몇 번씩 다른 거 써보다가도 다시 돌아오 게 되는 장미향이다.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 블랙 쿠션
일치하는 상품명 목록:
1. 블랙 쿠션 SPF34/PA++
2. 블랙 쿠션 SPF34/PA++ 15g
3. 헤라X39ETC 엑스커베이션 블랙 쿠션 21N1 15g x 2
4. 블랙 쿠션 제로-비티 핏 퍼프 2EA
상품을 선택하세요 (번호 입력): 1
블랙 쿠션 SPF34/PA++의 전체 리뷰 요약: 전에 체험단 이벤트로 27n1컬러를 처음 만났는데 21호 쓰는 제게는 많이 어두워서 신랑 쓰라고 줬더니 그 뒤로는 헤라가 갓이긴 갓이라면서 은근 계속 쓰고 싶다는 눈치였고 이번에 리뉴얼 되었는지 가격이 5.5만에서 6만으로 변경되기도 했고 사은품 행사가 좋아서 블랙쿠션이랑 헤라 아쿠아볼릭 3종과 같이 구매했는데 키트와 사은품이 정말 다양했다.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): q
챗봇을 종료합니다.


In [13]:
!pip install konlpy

In [14]:
from konlpy.tag import Okt
from collections import Counter

In [13]:

#def keyword_bot(df):
#    while True:
#        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
#        if title.lower() in ['q', 'quit']:
#            print("챗봇을 종료합니다.")
#            break
#        matching_titles = df[df['상품명'].str.contains(title, case=False)]['상품명'].unique().tolist()
#        if not matching_titles:
#            print("해당 상품이 없습니다.")
#            continue
#        print("일치하는 상품명 목록:")
#        for idx, matching_title in enumerate(matching_titles, start=1):
#            print(f"{idx}. {matching_title}")
#        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
#        selected_title = matching_titles[selected_idx - 1]
#        keyword(selected_title)  # 상품명을 함수에 전달하여 실행

#def keyword(title):
#    texts = df[df['상품명'] == title]['리뷰'].to_list()
#    okt = Okt()
#    nouns_list = []
#    for text in texts:
#        nouns = okt.nouns(text)
#        nouns_list.extend(nouns)

#    count = Counter(nouns_list)

    # 명사 빈도 카운트
    # 상위 100개
#    noun_list = count.most_common(100)
#    for v in noun_list:
#        if any(keyword in v[0] for keyword in ['지성','수부지', '건성', '복합성','여드름','잡티','피부톤','탄력','피지','모공','블랙헤드','민감성','봄웜','웜톤','봄웜톤','봄라이트','봄라','봄브라이트','여쿨','여름쿨톤','여라','여름라이트','여름뮤트','여뮽','뮤트','뮤트톤','여름브라이트','가을웜톤','가을뮤트','갈딥','가을딥','갈웜','겨쿨','겨울브라이트','겨울딥','쿨톤']):
#            print("이 제품은 "+v[0]+" 타입에게 추천하는 제품입니다.")

In [26]:
# Okt 객체 초기화
okt = Okt()
# 상품분류별 키워드 리스트 정의
skincare_keywords = ['지성', '수부지', '건성', '복합성', '여드름', '잡티', '피부톤', '탄력', '피지', '모공', '블랙헤드', '민감성']
makeup_keywords = ['복합성','지성','건성','잡티','커버력','촉촉','보송','봄웜', '웜톤', '봄웜톤', '봄라이트', '봄라', '봄브라이트', '여쿨', '여름쿨톤', '여라', '여름라이트', '여름뮤트', '여뮽', '뮤트', '뮤트톤', '여름브라이트', '가을웜톤', '가을뮤트', '갈딥', '가을딥', '갈웜', '겨쿨', '겨울브라이트', '겨울딥', '쿨톤']
perfume_keywords = ['플로럴','꽃','우디','우드','나무','머스크','비누','시트러스','장미','풀','그리너리','과일','프루티','코코넛','무화과','파우더리','봄','여름','가을','겨울','구어망드','달달','초콜릿','솜사탕','숲','민트','사과','복숭아','빨래','시원']
food_keywords = ['다이어트','붓기','유산균','면역','소화','피곤','활력','피로회복','숙취','관절','뼈','근육','피부']
thing_keywords = ['밀착력','쫀쫀','내구성','촉촉','뽀송','부들','말랑','품질','고급','깨끗','편리','휴대','간편','편안','그립감','탄력','정교','따뜻','유용','가볍','시원']
# 함수 정의
def keyword_bot(df):
    while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        matching_titles = df[df['상품명'].str.contains(title, case=False)]['상품명'].unique().tolist()
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")
        selected_idx = input("상품을 선택하세요 (번호 입력, 종료하려면 'q' 또는 'quit'): ")
        if selected_idx.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        try:
            selected_idx = int(selected_idx)
        except ValueError:
            print("유효한 번호를 입력해주세요.")
            continue
        selected_title = matching_titles[selected_idx - 1]
        selected_category = df[df['상품명'] == selected_title]['상품분류'].iloc[0]
        # 카테고리별 출력
        if selected_category == '스킨케어':
            selected_keywords = skincare_keywords
            message_type = '타입에게 추천하는 제품'
        elif selected_category == '메이크업':
            selected_keywords = makeup_keywords
            message_type = '용 제품'
        elif selected_category == '향수':
            selected_keywords = perfume_keywords
            message_type = '향이 나는 향수'
        elif selected_category == "뷰티푸드":
          selected_keywords = food_keywords
          message_type = '에 효과가 있는 제품'
        elif selected_category == '소품도구':
          selected_keywords = thing_keywords
          message_type = '이라는 특징이 있는 제품'
        else:
            selected_keywords = []
            message_type = '이 제품은 특별한 키워드가 없습니다'
        texts = df[df['상품명'] == selected_title]['리뷰'].to_list()
        nouns_list = []
        for text in texts:
            nouns = okt.nouns(text)
            nouns_list.extend(nouns)
        count = Counter(nouns_list)
        noun_list = count.most_common(100)
        for v in noun_list:
            if any(keyword in v[0] for keyword in selected_keywords):
                print(f"이 제품은 {v[0]} {message_type}입니다.")

In [27]:
keyword_bot(df)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 프로바이오틱스
일치하는 상품명 목록:
1. 프로바이오틱스 세라마이드 에멀전
2. 프로바이오틱스 세라마이드 스킨소프너
3. 프로바이오틱스 세라마이드 인텐스 크림
4. 프로바이오틱스 세라마이드 크림
5. 프로바이오틱스 세라마이드 2종
6. 프로바이오틱스 골드
상품을 선택하세요 (번호 입력, 종료하려면 'q' 또는 'quit'): 6
이 제품은 유산균 에 효과가 있는 제품입니다.
이 제품은 면역 에 효과가 있는 제품입니다.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 클린
일치하는 상품명 목록:
1. 포어 클린 토너
2. 클린테라피 파우더 마스크 홀리추얼 클린테라피 파우더 마스크
3. 클린 웜 코튼 EDP
4. 클린 퓨어솝 EDP
5. 클린 퓨어솝 EDP 1+1
상품을 선택하세요 (번호 입력, 종료하려면 'q' 또는 'quit'): 3
이 제품은 비누 향이 나는 향수입니다.
이 제품은 여름 향이 나는 향수입니다.
이 제품은 봄 향이 나는 향수입니다.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): q
챗봇을 종료합니다.


In [ ]:
6